In [ ]:
# Import necessary libraries
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from optimum.bettertransformer import BetterTransformer
import bitsandbytes as bnb

In [ ]:
# Set device to GPU if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Load tokenizer and model
model_name = "your-model-name"  # Replace with the model name or path to your custom model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with memory-efficient settings
try:
    from bitsandbytes import quantization
    
    # Load model with 4-bit quantization for memory efficiency
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        load_in_4bit=True,  
        quantization_config=quantization.Linear8bitLtConfig(), 
        device_map="auto"   
    ).to(device)
except ImportError as e:
    # Fallback if bitsandbytes is not available or GPU is not supported
    print("Error: bitsandbytes module import failed or missing GPU support.")
    print(e)
    
    model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)


In [ ]:
# Load and prepare the dataset
dataset = load_dataset("your-dataset-name")  # Replace with the dataset name or path

# Define a function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# Optionally remove unnecessary columns if needed
# tokenized_datasets = tokenized_datasets.remove_columns(["X", "Y", "Z"])
tokenized_datasets.set_format("torch")  # Convert dataset to PyTorch tensors

# Configure LoRA (Low-Rank Adaptation) with PEFT (Parameter-Efficient Fine-Tuning)
lora_config = LoraConfig(
    r=8,  
    lora_alpha=32, 
    target_modules=["classifier"], 
    lora_dropout=0.1,
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [ ]:
# Optimize the model for training
model = BetterTransformer.transform(model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    num_train_epochs=3, 
    save_steps=10_000, 
    save_total_limit=2, 
    evaluation_strategy="steps",
    eval_steps=1_000, 
    logging_dir="./logs",  
    no_cuda=False
)

# Create a Trainer instance with the model and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],  # Training dataset
    eval_dataset=tokenized_datasets["validation"]   # Validation dataset
)



In [ ]:
# Start training the model
trainer.train()

In [ ]:
# Revert the model to its original state (undo BetterTransformer optimizations)
model = BetterTransformer.reverse(model)

# Save the fine-tuned model and tokenizer
model.save_pretrained("./name")  # Save the model to the specified directory
tokenizer.save_pretrained("./name")  # Save the tokenizer to the specified directory

print("Training completed successfully.")